In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME      PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
mordehai  GCE       4                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0

In [3]:
import pyspark
import sys
import math
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
import gcsfs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 14 00:17 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext


In [6]:
bucket_name = '208906255' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.startswith("multi"):  # Reads only the data files and not the indices
        paths.append(full_path+b.name)

In [7]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2023-01-14 00:53:45--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  38.8MB/s    in 0.3s    

2023-01-14 00:53:45 (38.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip


In [9]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 2JamtMFX2doKbULQS64ce1JpX0e_6QWt8J6xDtSW7vj8XH8kP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
!pip -q install flask_restful

In [11]:
!pip install itsdangerous==2.0.1

In [ ]:
# %cd -q /home/dataproc
# !ls search_frontend_gcp.py

In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/search_frontend.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
sc.addFile("/home/dataproc/updated_inverted_index.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
# you need to upload your implementation of search_app.py
import updated_inverted_index
import search_frontend as se
se.load_global()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# # you need to upload your implementation of search_app.py
# import old_search_frontend as ose
# ose.load_global()

In [14]:
import importlib
importlib.reload(se)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'search_frontend' from '/hadoop/spark/tmp/spark-c273e6a7-1b05-431e-8830-b77f524dc8b1/userFiles-504176aa-15b5-4fd6-8797-e25acf187e5b/search_frontend.py'>

In [ ]:
# !gsutil -m cp gs://{bucket_name}/title_index/* "/home/dataproc"

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://25b1-34-68-92-184.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Jan/2023 00:55:08] "GET /search_title?query=best+marvel+movie HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:09] "GET /search_title?query=How+do+kids+come+to+world%3F HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:09] "GET /search_title?query=Information+retrieval HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:09] "GET /search_title?query=LinkedIn HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:09] "GET /search_title?query=How+to+make+coffee%3F HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:09] "GET /search_title?query=Ritalin HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:10] "GET /search_title?query=How+to+make+wine+at+home%3F HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:10] "GET /search_title?query=Most+expensive+city+in+the+world HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:11] "GET /search_title?query=India HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 00:55:11] "GET /search_title?query=how+to+make+money+fast%3F HTTP/1.1" 200 -
127.0.0.1 - - [14/J